# The cash matching problem

**Referencia:**
- Investment Science, David G. Luenberger, 2014.

## 0. Motivación

1. La clase pasada vimos una introducción a programación lineal. 
 - <font color=blue>Básicamente, ¿qué es?</font>
 - <font color=blue>¿Cómo son las funciones que se optimizan?</font>
 - <font color=blue>De acuerdo a lo anterior, ¿qué tipo de problemas se resuelven con LP?</font>
2. La optimización de portafolios es un problema de asignación de capital en diferentes opciones de inversión.
 - Si se tienen activos riesgosos, el problema de optimización que surge es uno de QP.
 - Con activos de renta fija (libres de riesgo, flujo de efectivo conocido en cantidad y en tiempo), el problema que surge es uno de LP, como vimos la clase pasada.
3. Aún así, dentro de la optimización de portafolios constituidos por activos libres de riesgo, hay varios subproblemas clásicos que se pueden plantear, uno de ellos es el **cash matching problem**

## 1. Conceptos básicos

Suponga que usted tiene unas obligaciones monetarias futuras (por ejemplo, dichas obligaciones podrían representar los pagos de anualidad de un plan de retiro, seguro de carro, consumo anual estimado del agua, entre otros).

Queremos invertir ahora, de forma que esas obligaciones se puedan pagar una vez se generen los cobros.

Para ello, planeamos comprar bonos con diferente madurez y utilizar los pagos cupón y pago final para atender las obligaciones.

En su forma más básica, se diseña un portafolio que, sin inversión adicional, provee el efectivo necesario cuando se requiera.

Matemáticamente, podemos formular este problema de la siguiente manera...

\begin{equation}
\begin{array}{ll}
\min_{x_1,\dots,x_n} & \sum_{j=1}^{m} p_jx_j \\
\text{s. a. }        & \sum_{j=1}^{m}c_{i,j}x_j\geq y_i \text{ para } 1\leq i\leq n \\
                     & x_j \text{ para } 1\leq j\leq m,
\end{array}
\end{equation}
donde:
- $x_j$ es la cantidad del bono $j$ a ser comprada ($j=1,2,\dots,m$),
- $p_j$ es el precio del bono $j$,
- $y_i$ es la obligación en el periodo $i$ ($i=1,2,\dots,n$), y
- $c_{i,j}$ es el cupón del bono $j$ en el periodo $i$.

## 2. Ejemplo básico

- Queremos atender obligaciones de efectivo sobre un periodo de 6 años.
- Para este fin seleccionamos 10 bonos (y para simplicidad toda la contabilidad se lleva anual). 
- La estructura de flujo de efectivo de cada bono se muestra en la columna correspondiente de la tabla.
- Los precios de cada bono están dados en el vector $p$.
- Las obligaciones están dadas en el vector $y$.
- Se supone que se pueden comprar fracciones de bono.

In [1]:
import pandas as pd
import numpy as np

In [2]:
table = pd.DataFrame(index=np.arange(6)+1, columns=np.arange(10)+1)
table.iloc[:,0] = [10,10,10,10,10,110]
table.iloc[:,1] = [7,7,7,7,7,107]
table.iloc[:,2] = [8,8,8,8,8,108]
table.iloc[:,3] = [6,6,6,6,106,0]
table.iloc[:,4] = [7,7,7,7,107,0]
table.iloc[:,5] = [5,5,5,105,0,0]
table.iloc[:,6] = [10,10,110,0,0,0]
table.iloc[:,7] = [8,8,108,0,0,0]
table.iloc[:,8] = [7,107,0,0,0,0]
table.iloc[:,9] = [100,0,0,0,0,0]
table

1    2    3    4    5    6    7    8    9    10
1   10    7    8    6    7    5   10    8    7  100
2   10    7    8    6    7    5   10    8  107    0
3   10    7    8    6    7    5  110  108    0    0
4   10    7    8    6    7  105    0    0    0    0
5   10    7    8  106  107    0    0    0    0    0
6  110  107  108    0    0    0    0    0    0    0

In [6]:
C = table.values
p = np.array([109, 94.8, 99.5, 93.1, 97.2, 92.9, 110, 104, 102, 95.2])
y = np.array([100, 200, 800, 100, 800, 1200])

Identificar las entradas de la función linprog y resolver...

In [3]:
from scipy.optimize import linprog

In [7]:
res_cfm1 = linprog(p, A_ub=-C, b_ub=-y)
res_cfm1

     fun: 2381.1388298203824
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([ 71.74111374,   0.        ,   0.        ,  19.34403104,
         0.        ,   0.        ])
  status: 0
 success: True
       x: array([  0.        ,  11.21495327,   0.        ,   6.80655969,
         0.        ,   0.        ,   0.        ,   6.30237008,
         0.28258886,   0.        ])

In [8]:
res_cfm1.x

array([  0.        ,  11.21495327,   0.        ,   6.80655969,
         0.        ,   0.        ,   0.        ,   6.30237008,
         0.28258886,   0.        ])

In [12]:
C.dot(res_cfm1.x)

array([  171.74111374,   200.        ,   800.        ,   119.34403104,
         800.        ,  1200.        ])

In [10]:
y

array([ 100,  200,  800,  100,  800, 1200])

<font color=red> **¿Qué particularidades se encuentran?** </font>

## 3. Ejemplo

- La compañía XYZ desea invertir en un conjunto de 7 bonos para garantizar el cumplimient de obligaciones sobre los siguientes 14 años.
- La estructura de flujo de efectivo de cada bono se muestra en la columna correspondiente de la tabla.
- Los precios de cada bono están dados en el vector $p$.
- Las obligaciones están dadas en el vector $y$.
- Además de los bonos, se tiene una cuenta de ahorro cuya tasa de interés anual es del $3\%$.
- Cualquier excedente de efectivo después de cumplir los requerimientos de periodo se invierten en la cuenta de efectivo. 
- La meta es determinar el portafolio de mínimo valor hoy para cumplir con todas las obligaciones
- Se supone que se pueden comprar fracciones de bono.

In [13]:
table2 = pd.DataFrame(index=np.arange(14)+1, columns=np.arange(7)+1)
table2.iloc[:,0] = [60,60,60,60,1060,0,0,0,0,0,0,0,0,0]
table2.iloc[:,1] = [65,65,65,65,65,65,65,65,65,65,1065,0,0,0]
table2.iloc[:,2] = [75,75,75,75,75,75,75,75,75,75,75,75,75,1075]
table2.iloc[:,3] = [55,55,55,55,55,55,55,55,55,55,55,55,1055,0]
table2.iloc[:,4] = [80,80,80,80,80,80,80,80,80,1080,0,0,0,0]
table2.iloc[:,5] = [45,45,45,45,45,45,45,45,45,45,45,45,45,1045]
table2.iloc[:,6] = [100,100,100,100,100,100,1100,0,0,0,0,0,0,0]
table2

1     2     3     4     5     6     7
1     60    65    75    55    80    45   100
2     60    65    75    55    80    45   100
3     60    65    75    55    80    45   100
4     60    65    75    55    80    45   100
5   1060    65    75    55    80    45   100
6      0    65    75    55    80    45   100
7      0    65    75    55    80    45  1100
8      0    65    75    55    80    45     0
9      0    65    75    55    80    45     0
10     0    65    75    55  1080    45     0
11     0  1065    75    55     0    45     0
12     0     0    75    55     0    45     0
13     0     0    75  1055     0    45     0
14     0     0  1075     0     0  1045     0

In [14]:
C = table2.values
p = np.array([990, 1000, 1300, 1100, 1200, 990, 1250])
y = np.array([12000, 14000, 15000, 16000, 18000, 20000, 21000, 22000, 24000, 25000, 30000, 31000, 31000, 31000])

Formular el problema, identificar las entradas a la función linprog y resolver

In [20]:
c = np.concatenate((np.array([1]), np.zeros((14,))), axis=0)
c = np.concatenate((c, p), axis=0)
c

array([  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         9.90000000e+02,   1.00000000e+03,   1.30000000e+03,
         1.10000000e+03,   1.20000000e+03,   9.90000000e+02,
         1.25000000e+03])

In [35]:
r = 0.03
Aeq1 = np.concatenate(((1+r)*np.eye(14), np.zeros((14,1))), axis = 1)
Aeq2 = np.concatenate((np.zeros((14,1)),-np.eye(14)), axis = 1)
Aeq = np.concatenate((Aeq1+Aeq2, C), axis=1)

In [39]:
res_cfm2 = linprog(c, A_eq=Aeq, b_eq=y, method='interior-point')
res_cfm2

     con: array([ 0.00076515,  0.00089798,  0.0009644 ,  0.00103082,  0.00109724,
        0.00130048,  0.00130048,  0.00143996,  0.0015728 ,  0.0015728 ,
        0.00191021,  0.00204736,  0.00198094,  0.00191818])
     fun: 193327.06954183165
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([  4.76625036e+03,   5.82885786e+03,   4.92334371e+03,
         2.99066421e+03,   4.39291519e-03,   7.61365993e+03,
         4.81442378e-02,   2.77121890e+04,   1.50461171e+04,
         6.32320247e-02,   5.86688103e-03,   8.76869312e+04,
         5.93175420e+04,   3.00970749e+04,   1.34496617e-03,
         1.26940351e+01,   1.10504152e+02,   4.19518459e-06,
         3.78954935e-06,   1.64973781e+01,   7.44392931e-06,
         3.65541610e+01])

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>